# Production éolienne

## Import des librairies

In [1]:
import pandas as pd
import numpy as np
import requests
from datetime import date, datetime, timezone

## Chargement des données
- prod_eolienne.csv

In [2]:
df = pd.read_csv('data/prod_eolienne.csv')
display(df.dtypes)
display(df)


date              object
prod_eolienne    float64
dtype: object

,date,prod_eolienne
0,2016-09-01,100.000000
1,2016-09-02,100.000000
2,2016-09-03,100.000000
3,2016-09-04,96.844120
4,2016-09-05,97.088424
...,...,...
3303,2025-09-25,72.259819
3304,2025-09-26,52.211649
3305,2025-09-27,38.421115
3306,2025-09-28,55.810392


## En résumé
- 3308 entrées
- 2 colonnes 


In [3]:
display(df.dtypes)
display(df.describe())


date              object
prod_eolienne    float64
dtype: object

,prod_eolienne
count,3308.000000
mean,60.261740
std,27.466505
min,-48.997601
25%,44.321585
50%,61.730794
75%,76.859869
max,833.516984


### Valeurs manquantes et aberrantes

In [4]:
# Valeurs manquantes
df_missing = df.isnull().sum()
display(df_missing)

# Valeur extrême

df_weird = df.sort_values(by=['prod_eolienne'], ascending=True)
display(min(df_weird['prod_eolienne']))
display(df_weird)

df_max = df.sort_values(by=['prod_eolienne'], ascending=False)

print(df_max)

date             0
prod_eolienne    0
dtype: int64

-48.99760076569133

,date,prod_eolienne
2794,2024-05-04,-48.997601
2354,2023-02-19,7.077903
3262,2025-08-15,7.656243
2353,2023-02-18,8.626531
3261,2025-08-14,9.223880
...,...,...
1041,2019-07-17,100.000000
1043,2019-07-19,100.000000
1275,2020-03-07,100.000000
1014,2019-06-19,666.206615


            date  prod_eolienne
1002  2019-06-07     833.516984
1014  2019-06-19     666.206615
578   2018-04-07     100.000000
0     2016-09-01     100.000000
583   2018-04-12     100.000000
...          ...            ...
3261  2025-08-14       9.223880
2353  2023-02-18       8.626531
3262  2025-08-15       7.656243
2354  2023-02-19       7.077903
2794  2024-05-04     -48.997601

[3308 rows x 2 columns]


### Traitement des valeurs aberrantes
- Si valeur < 0 -> passage à une valeur absolue
- Si valeur > 100 -> passage à une valeur  NULL

In [5]:
# Fonction pour traiter les valeurs aberrantes

def clean_value(number):
    if number < 0:
        number = abs(number)
    if number > 100:
        return None
    return number

df['prod_eolienne'] = df['prod_eolienne'].apply(clean_value)
df = df.dropna()

display(df.isnull().sum())
display(df.describe())
display(df.sort_values(by=['prod_eolienne']))

date             0
prod_eolienne    0
dtype: int64

,prod_eolienne
count,3306.000000
mean,59.874200
std,21.427358
min,7.077903
25%,44.331044
50%,61.704084
75%,76.830423
max,100.000000


,date,prod_eolienne
2354,2023-02-19,7.077903
3262,2025-08-15,7.656243
2353,2023-02-18,8.626531
3261,2025-08-14,9.223880
3036,2025-01-01,9.357384
...,...,...
1987,2022-02-17,100.000000
1607,2021-02-02,100.000000
1905,2021-11-27,100.000000
1,2016-09-02,100.000000


### Conversion de la colonne 'date' au format datetime

In [6]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

display(df.dtypes)

/tmp/ipykernel_43466/1996876030.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')


date             datetime64[ns]
prod_eolienne           float64
dtype: object

# Paramètres influençant la production éolienne
- Vitesse moyenne du vent (10m)
- Direction du vent
- Température
- Pression atmosphérique (niveau de la mer)

Données récoltées depuis l'API open-meteo

In [7]:
# Chargement
df_param = pd.read_csv('data/param_eolienne.csv')


# Mise au format datetime
df_param['time'] = pd.to_datetime(df_param['time'])
# Renommage colonne date (time -> date)
df_param.rename(columns={'time': 'date'}, inplace=True)

print(df_param)

           date  winddirection_10m_dominant (°)  wind_speed_10m_mean (km/h)  \
0    2016-09-01                             269                        12.0   
1    2016-09-02                             234                        12.6   
2    2016-09-03                             237                        11.5   
3    2016-09-04                             270                        11.6   
4    2016-09-05                             307                        17.6   
...         ...                             ...                         ...   
3312 2025-09-26                             306                         6.4   
3313 2025-09-27                             355                         6.3   
3314 2025-09-28                               2                         5.4   
3315 2025-09-29                             340                         4.4   
3316 2025-09-30                             357                         7.3   

      temperature_2m_mean (°C)  pressure_msl_mean (

In [8]:
# Résumé
display('Résumé : ',df_param.describe())

# Types de données

display('Types des données : ',df_param.dtypes)

## Missing values

display('Valeurs manquantes : ', df_param.isnull().sum())

## Doublons

display('Doublons :', df_param.duplicated().sum())

'Résumé : '

,date,winddirection_10m_dominant (°),wind_speed_10m_mean (km/h),temperature_2m_mean (°C),pressure_msl_mean (hPa)
count,3317,3317.000000,3317.000000,3317.000000,3317.000000
mean,2021-03-17 00:00:00,214.220380,12.146367,15.574977,1016.976485
min,2016-09-01 00:00:00,0.000000,3.500000,-2.200000,987.900000
25%,2018-12-09 00:00:00,113.000000,8.600000,10.000000,1013.300000
50%,2021-03-17 00:00:00,268.000000,10.900000,15.000000,1016.700000
75%,2023-06-24 00:00:00,316.000000,14.700000,21.400000,1020.800000
max,2025-09-30 00:00:00,360.000000,43.100000,33.600000,1038.400000
std,NaN,119.203858,4.924862,6.928749,6.927902


'Types des données : '

date                              datetime64[ns]
winddirection_10m_dominant (°)             int64
wind_speed_10m_mean (km/h)               float64
temperature_2m_mean (°C)                 float64
pressure_msl_mean (hPa)                  float64
dtype: object

'Valeurs manquantes : '

date                              0
winddirection_10m_dominant (°)    0
wind_speed_10m_mean (km/h)        0
temperature_2m_mean (°C)          0
pressure_msl_mean (hPa)           0
dtype: int64

'Doublons :'

np.int64(0)

## Encodage des données de direction du vent
- Afin de signifier l'aspect 'circulaire' de ce type de données, il est nécessaire d'encoder ces dernières.
- Transformer une valeur d'angle en deux valeurs : sin (nord-sud) et cos (est-ouest)
- Ces deux nouvelles valeurs seront des features indépendantes pour l'entrainement du modèle

In [10]:
# Mise en fonction de la conversion de la direction du vent

def convert_wind_dir(df: pd.DataFrame, col_name: str='daily_winddirection_10m_dominant (°)') -> pd.DataFrame:
    """
    Encode une colonne d'angle en degrés (0-360) sous forme trigonométrique (sin, cos).
    
    Args:
        df (pd.DataFrame): DataFrame contenant la colonne d'angle en degrés.
        col_name (str): Nom de la colonne contenant la direction du vent en degrés.
    
    Returns:
        pd.DataFrame: DataFrame avec deux nouvelles colonnes ('wind_dir_sin', 'wind_dir_cos'),
                      et sans la colonne d'origine en degrés ni sa version en radians.
    """
    df = df.copy()
    # renommage
    df = df.rename(columns={col_name: 'wind_direction'})
    # conversion en radians
    df['wind_direction_rad'] = np.deg2rad(df['wind_direction'])
    # encodage trigo
    df['wind_dir_sin'] = np.sin(df['wind_direction_rad'])
    df['wind_dir_cos'] = np.cos(df['wind_direction_rad'])
    # suppression colonnes précédentes
    df = df.drop(columns=['wind_direction', 'wind_direction_rad'])
    return df
    

    

## Comparaison des dataframes
- En vue d'une fusion, on vérifie les correspondances au niveau des dates
- Le but étant d'avoir un dataframe ayant le label/target et les variables qui seront utilisées pour l'entrainement d'un modèle

In [11]:
## Comparaison des 2 dataframes

# Suppresion de la date en trop (2025-09-30)
df_param = df_param[df_param['date'] != pd.to_datetime("2025-09-30")]

display('Prod : ',df['date'].min(), df['date'].max())
display('Météo : ',df_param['date'].min(), df_param['date'].max())

## Verif des doublons 

display(df['date'].nunique())
display(df_param['date'].nunique())

'Prod : '

Timestamp('2016-09-01 00:00:00')

Timestamp('2025-09-29 00:00:00')

'Météo : '

Timestamp('2016-09-01 00:00:00')

Timestamp('2025-09-29 00:00:00')

3306

3316

In [12]:
## Recherches des dates sans correspondances
date_diff = set(df_param['date']) - set(df['date'])
display(sorted(list(date_diff)))

[Timestamp('2016-09-22 00:00:00'),
 Timestamp('2016-09-23 00:00:00'),
 Timestamp('2016-09-24 00:00:00'),
 Timestamp('2016-09-25 00:00:00'),
 Timestamp('2017-05-17 00:00:00'),
 Timestamp('2018-04-15 00:00:00'),
 Timestamp('2018-04-16 00:00:00'),
 Timestamp('2019-06-07 00:00:00'),
 Timestamp('2019-06-19 00:00:00'),
 Timestamp('2019-06-23 00:00:00')]

In [13]:
## Fusion des dataframes

df_merged = pd.merge(
    df, df_param,
    on='date', how='inner'
)

## Classe - Fonctions spécifiques
### Traitement & Nettoyage
- Encoder les valeurs de direction du vent (valeur en degré -> radians) 
- Valeur de production située dans l'intervalle [0, 100] (nettoyage)
- Encoder les dates (object -> datetime)
### Sauvegarde dans BDD
- Doit contenir la date (datetime), production_eolienne, 